## SRP055882/GSE66582

**paper:** [PMID: 27309802](https://www.nature.com/articles/nature18606) - The landscape of accessible chromatin in mammalian preimplantation embryos, 2016

**date, curator:** 2024-10-22, Sara Carsanaro

**resources**

**notes**
* removed libraries that are not healthy wild type: 
    * mESC (cell culture) - SRX902563, SRX902562 
    * Nr5a2 siRNA knockdown samples - SRX1424874, SRX1424873, SRX1424872, SRX1424871   
    * Gata4 siRNA knockdown samples - SRX902570, SRX902569, SRX902568, SRX902567
* anaId notes
    * used CL terms when the anatomical entitiy is a cell (zygote, MII oocyte)
* stage name note
    * added time in hours for each stage according to what is stated in the methods section
* sex: added F for MII oocyte b/c we know this came from a female, all others are NA

### annotation summary
run these when annotation is complete

In [24]:
# cell type annotations
display_df(cell_df_complete)

,infoCellType_abInitio,notes_cell,cellTypeId_,cellTypeName_,cellTypeAnnotationStatus_,cell_type_new,comment_cell
0,2-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,
1,4-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,
2,8-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,
3,Blastocyst,"using this general term for blastocyst cell, as we did in annotation of SRP020490. not using blastoderm cell because per the blastocyst definition: he mammalian blastocyst is a hollow ball of cells containing two cell types, the inner cell mass and the trophectoderm",CL:0002321,embryonic cell (metazoa),missing child term,,
4,Inner cell mass,Any native cell that is part of a inner cell mass.,CL:2000020,inner cell mass cell,perfect match,,
5,MII oocyte,"egg cell definition: A female gamete where meiosis has progressed to metaphase II and is able to participate in fertilization. MII oocyte is a metaphase II stage oocyte, therefore this annotation is appropriate.",CL:0000025,egg cell,perfect match,,
6,Zygote,,CL:0000365,animal zygote,perfect match,,
7,early 2-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,


In [7]:
# organ annotations
display_df(organ_df_complete)

,infoOrgan,notes_anat,anatId_,anatName_,anatAnnotationStatus_,infoOrgan_new,comment_anat
0,Inner cell mass,,UBERON:0000087,inner cell mass,perfect match,,
1,MII oocyte,"egg cell definition: A female gamete where meiosis has progressed to metaphase II and is able to participate in fertilization. MII oocyte is a metaphase II stage oocyte, therefore this annotation is appropriate.",CL:0000025,egg cell,perfect match,,
2,Preimplantation embryo: 2-cell,,UBERON:0019249,2-cell stage embryo,perfect match,,
3,Preimplantation embryo: 4-cell,,UBERON:0019251,4-cell stage embryo,perfect match,,
4,Preimplantation embryo: 8-cell,,UBERON:0019252,8-cell stage embryo,perfect match,,
5,Preimplantation embryo: Blastocyst,,UBERON:0000358,blastocyst,perfect match,,
6,Preimplantation embryo: Zygote,using the CL term here because there is no UBERON term for a single cell zygote,CL:0000365,animal zygote,perfect match,,
7,Preimplantation embryo: early 2-cell,,UBERON:0019249,2-cell stage embryo,missing child term,,


In [23]:
# stage annotations
display_df(stage_df_complete)

,infoStage,notes_stage,stageId_,stageName_,stageAnnotationStatus_,infoStage_new,comment_stage
0,2-cell,Embryonic cleavage stage defined by a dividing 2-4 cells egg,MmusDv:0000005,Theiler stage 02,missing child term,2-cell (39-42h),
1,4-cell,Embryonic cleavage stage defined by a dividing 2-4 cells egg,MmusDv:0000005,Theiler stage 02,missing child term,4-cell (54-56h),
2,8-cell,"Embryonic cleavage stage defined by a dividing 4-16 cells egg, early to fully compacted morula.",MmusDv:0000006,Theiler stage 03,missing child term,8-cell (68-70h),
3,Blastocyst,Embryonic blastula stage defined by a zona free blastocyst (zona pellucida absent). Embryonic age 4 dpc (day post coitum).,MmusDv:0000009,Theiler stage 05,missing child term,Blastocyst (92-94h),
4,Inner cell mass,"Embryonic stage that is an early stage of embryonic development in animals and is produced by cleavage of a fertilized ovum, with formation of a central fluid-filled cavity called the blastocoel.",MmusDv:0000007,blastula stage,other,Inner cell mass,
5,MII oocyte,"mature stage definition: Mouse developmental stage that refers to a sexually mature adult mouse which is over 6 weeks old. Prime adult is a subset of mature stage, which is applicable here because we know the approximate age of the female",MmusDv:0000136,prime adult stage,other,MII oocyte (taken from 5- to 6-week-old female),
6,Zygote,Embryonic stage defined by a one-cell embryo (fertilised egg) with zona pellucida present,MmusDv:0000003,Theiler stage 01,perfect match,Zygote,
7,early 2-cell,Embryonic cleavage stage defined by a dividing 2-4 cells egg,MmusDv:0000005,Theiler stage 02,missing child term,early 2-cell (30_h),


### set variables, import packages, define functions

In [1]:
experiment_id = "SRP055882"

path_to_create_exp_script = "/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py" 
path_to_barcodes_script = "/Users/scarsana/Desktop/git/scRNA-Seq/scripts/create_scRNASeq_barcode_v2.py"
experiment_type = "scRNA"

path_to_output_main = "/Users/scarsana/Desktop/git/expression-annotations/Notebooks/scRNA/" 
path_to_output = "{}{}/".format(path_to_output_main, experiment_id)
library_path_from_script = "{}scRNASeqLibrary_{}.tsv".format(path_to_output, experiment_id)
experiment_path_from_script = "{}scRNASeqExperiment_{}.tsv".format(path_to_output, experiment_id)
barcode_path = "{}scRNASeq_barcode_{}.tsv".format(path_to_output, experiment_id)
barcode_to_add_path = "{}complete_scRNASeq_barcode_{}.tsv".format(path_to_output, experiment_id)
library_to_add_path = "{}complete_scRNASeqLibrary_{}.tsv".format(path_to_output, experiment_id)
experiment_to_add_path = "{}complete_RNASeqExperiment_{}.tsv".format(path_to_output, experiment_id)
script_file = "{}.ipynb".format(experiment_id)
commit_message_exp = '"adding annotated scRNA experiment {}"'.format(experiment_id)
commit_message_py = '"adding annotation files for {} to notebook folder"'.format(experiment_id)



## to add to git
path_to_git_annotations = "/Users/scarsana/Desktop/git/expression-annotations/scRNA_Seq/"
git_library_path = "{}scRNASeqLibrary_merged.tsv".format(path_to_git_annotations)
git_experiment_path = "{}scRNASeqExperiment.tsv".format(path_to_git_annotations)
git_barcode_path = "{}scRNASeq_barcode_{}.tsv".format(path_to_git_annotations, experiment_id)

## columns
library_cols = ['#libraryId', 'experimentId', 'platform', 'SRSId', 'anatId', 'anatName', 'cellTypeId', 'cellTypeName', 'stageId', 'stageName', 'url_GSM', 'infoOrgan', 'infoCellType_abInitio', 'infoCellType_inferred', 'clusterId', 'clusterName', 'infoStage', 'anatAnnotationStatus', 'cellTypeAnnotationStatus', 'stageAnnotationStatus', 'sex', 'strain', 'genotype', 'speciesId', 'RNAseqTags', 'protocol', 'protocolType', 'lib_name', 'sampleName', 'comment', 'condition', 'annotatorId', 'lastModificationDate']
barcode_cols = ['barcode', 'cluster', 'library', 'experiment', 'tissue', 'cell_type', 'anatId_a_posteriori', 'anatName_a_posteriori', 'anat_a_posteriori_annotationStatus', 'cellTypeId', 'cellTypeName', 'cellTypeAnnotationStatus', 'name_Library', 'comments']

## annotation sheet paths
annotation_files_path = "{}annotation_files/".format(path_to_output)
import os
if not os.path.exists(annotation_files_path):
    os.makedirs(annotation_files_path)

info_anat_path = "{}info_anat.tsv".format(annotation_files_path)
info_anat_complete_path = "{}info_anat_complete.tsv".format(annotation_files_path)
info_stage_path = "{}info_dev_stage.tsv".format(annotation_files_path)
info_stage_complete_path = "{}info_dev_stage_complete.tsv".format(annotation_files_path)
info_celltype_path = "{}info_celltype.tsv".format(annotation_files_path)
info_celltype_complete_path = "{}info_celltype_complete.tsv".format(annotation_files_path)

## barcode file & matching library names - 10X only
path_to_barcode_file = "{}barcode_{}.tsv".format(path_to_output, experiment_id)
lib_names_path = "{}matching_lib_names_{}.tsv".format(path_to_output, experiment_id)



In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import csv

# displays df with the scrollbar next to the DataFrame
def display_df(df):
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
        df.style.to_html(index=False) + "</div>"))

# function that compares two columns in a dataframe and tells you which ones are not equal (case insensitive)
def compare_columns(df, col1, col2, return_col):
    compare_return = df[col1].str.lower() != df[col2].str.lower()  
    df.loc[compare_return, return_col] 
    if not any(compare_return):
        print("The two columns are equal (case insensitive)")
    else:
        print("The following rows are not equal: ")
        print(df.loc[compare_return, return_col])

# fixes formatting of file to match libreoffice settings/historic file format
def update_format(path):
    with open(path, 'r') as file:
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace("\t\"\"", "\t")
    # Write the file out again
    with open(path, 'w') as file:
        file.write(filedata)

# checks for duplicate values in a specific column and prints those values + the corresponding library id
def dup_check(df, column):
    duplicateCheck = df.duplicated(subset=[column], keep=False)
    duplicateCheck.sort_values(inplace=True)
    if duplicateCheck.unique().any() == False:
        print("no duplicate values in " + column)
    elif duplicateCheck.unique().any() == True and column != '#libraryId':
        dups = df[duplicateCheck].loc[:,['#libraryId', column]]
        df_dups = pd.DataFrame(dups)
        df_dups.sort_values(inplace=True, by=column)
        print(df_dups)
    elif duplicateCheck.unique().any() == True and column == '#libraryId':
        print(df[duplicateCheck].loc[:,['#libraryId']])

# prints all unique values in a specific column
def unique_sorted(df, column):
    unique = df[column].unique()
    unique.sort()
    print(unique)

### script

In [3]:
! python3 $path_to_create_exp_script $experiment_id $path_to_output $experiment_type

/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py:97: SyntaxWarning: invalid escape sequence '\('
  all_protoc = [w.replace('(', '\(') for w in all_protoc]
/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py:98: SyntaxWarning: invalid escape sequence '\)'
  all_protoc = [w.replace(')', '\)') for w in all_protoc] 
Be patient, it may take a few minutes.
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esummary.fcgi -query_key 1 -WebEnv MCID_671113ef8dbeacced20ae014 -retstart 0 -retmax 1 -db bioproject -version 2.0 -retmode xml -tool edirect -edirect 16.2 -edirect_os Darwin -email scarsana@SORGE42778
<ERROR>NCBI C++ Exception:&#xa;    Error: EUTILSAPP(CException::eInvalid) &quot;/pubmed_gen/rbuild/version/20240724/entrez/2.19/src/internal/eutils/eutilsapp/eutilsapp.cpp&quot;, line 77: ncbi::EUtilsException::EUtilsException() --- esummary: bioproject: pback220: CDTD_XSLT_Loader::GetData: dtd: Cannot open file: /entrez/local/bioproject/data/Bu

### library annnotations

In [4]:
library = pd.read_csv(library_path_from_script, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,,,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,,17/10/2024,"After removing the zona pellucida, embryonic cells were washed with PBS and incubated in lysis buffer. RNA-Seq libraries were prepared using Smart-Seq2 protocol with slightly modification",,,,Preimplantation embryo,,Preimplantation embryo: 8-cell,
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,,,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,,17/10/2024,"After removing the zona pellucida, embryonic cells were washed with PBS and incubated in lysis buffer. RNA-Seq libraries were prepared using Smart-Seq2 protocol with slightly modification",,,,Preimplantation embryo,,Preimplantation embryo: 8-cell,
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,,,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,,17/10/2024,"After removing the zona pellucida, embryonic cells were washed with PBS and incubated in lysis buffer. RNA-Seq libraries were prepared using Smart-Seq2 protocol with slightly modification",,,,Preimplantation embryo,,Preimplantation embryo: 8-cell,
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,,,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,,17/10/2024,"After removing the zona pellucida, embryonic cells were washed with PBS and incubated in lysis buffer. RNA-Seq libraries were prepared using Smart-Seq2 protocol with slightly modification",,,,Preimplantation embryo,,Preimplantation embryo: 8-cell,
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,,,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell,,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,,17/10/2024,"After removing the zona pellucida, embryonic cells were washed with PBS and incubated in lysis buffer. RNA-Seq libraries were prepared using Smart-Seq2 protocol with slightly modification",,,,Preimplantation embryo,,Preimplantation embryo: early 2-cell,
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,,,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell,,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,,17/10/2024,"After removing the zona pellucida, embryonic cells were washed with PBS and incubated in lysis buffer. RNA-Seq libraries were prepared using Smart-Seq2 protocol with slightly modification",,,,Preimplantation embryo,,Preimplantation embryo: early 2-cell,
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,,,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte,,,,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-

#### anatomical entity

In [5]:
infoOrgan = library['infoOrgan'].drop_duplicates().sort_values()
organ_df = pd.DataFrame(infoOrgan)
organ_df = organ_df.reindex(columns=[*organ_df.columns.tolist(), 'notes_anat', 'anatId_', 'anatName_', 'anatAnnotationStatus_', 'infoOrgan_new', 'comment_anat'], fill_value="")
organ_df.to_csv(info_anat_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
if not os.path.exists(info_anat_complete_path): 
    organ_df.to_csv(info_anat_complete_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

# options for anatAnnotationStatus: perfect match, missing child term, other

In [6]:
organ_df_complete = pd.read_csv(info_anat_complete_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
display_df(organ_df_complete)

,infoOrgan,notes_anat,anatId_,anatName_,anatAnnotationStatus_,infoOrgan_new,comment_anat
0,Inner cell mass,,UBERON:0000087,inner cell mass,perfect match,,
1,MII oocyte,"egg cell definition: A female gamete where meiosis has progressed to metaphase II and is able to participate in fertilization. MII oocyte is a metaphase II stage oocyte, therefore this annotation is appropriate.",CL:0000025,egg cell,perfect match,,
2,Preimplantation embryo: 2-cell,,UBERON:0019249,2-cell stage embryo,perfect match,,
3,Preimplantation embryo: 4-cell,,UBERON:0019251,4-cell stage embryo,perfect match,,
4,Preimplantation embryo: 8-cell,,UBERON:0019252,8-cell stage embryo,perfect match,,
5,Preimplantation embryo: Blastocyst,,UBERON:0000358,blastocyst,perfect match,,
6,Preimplantation embryo: Zygote,using the CL term here because there is no UBERON term for a single cell zygote,CL:0000365,animal zygote,perfect match,,
7,Preimplantation embryo: early 2-cell,,UBERON:0019249,2-cell stage embryo,missing child term,,


In [8]:
library = library.merge(organ_df_complete, left_on='infoOrgan', right_on='infoOrgan')

In [9]:
library['anatId'] = library['anatId_'].values
library['anatName'] = library['anatName_'].values
library['anatAnnotationStatus'] = library['anatAnnotationStatus_'].values

In [ ]:
## optional - update infoOrgan, add anat-specific comments 
#library['infoOrgan'] = library['infoOrgan_new'].values
#library['comment'] = library[['comment', 'comment_anat']].agg(' '.join, axis=1)

In [10]:
# reformat, save progress
library = library[library_cols]
display_df(library)
library.to_csv(library_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,perfect match,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,,17/10/2024
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,perfect match,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,,17/10/2024
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,perfect match,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,,17/10/2024
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell,perfect match,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,,17/10/2024
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell,missing child term,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,,17/10/2024
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell,missing child term,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,,17/10/2024
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte,perfect match,,,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,,17/10/2024
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte,perfect match,,,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,,17/10/2024
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBERON:0000358,blastocyst,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1625877,Preimplantation embryo: Blastocyst,Blastocyst,,,,Blastocyst,perfect match,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq blastocyst KD control rep3,"SAMN03389622,GSM1625877",,,,17/10/2024
9,SRX902565,SRP055882,Illumina HiSeq 1500,SRS865219,UBERON:0000358,blastocyst,,,,,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1625876,Preimplantation embryo: Blastocyst,Blastocyst,,,,Blastocyst,perfect match,,,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq blastocyst KD control rep2,"SAMN03389643,GSM1625876",,,,17/10/2024


#### stage
- [species specific developmental ontologies](https://github.com/obophenotype/developmental-stage-ontologies/tree/master/src)

In [11]:
infoStage = library['infoStage'].drop_duplicates().sort_values()
stage_df = pd.DataFrame(infoStage)
stage_df = stage_df.reindex(columns=[*stage_df.columns.tolist(), 'notes_stage', 'stageId_', 'stageName_', 'stageAnnotationStatus_', 'infoStage_new','comment_stage'], fill_value="")
stage_df.to_csv(info_stage_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
if not os.path.exists(info_stage_complete_path): 
    stage_df.to_csv(info_stage_complete_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

# options for stageAnnotationStatus: perfect match, missing child term, other

In [13]:
stage_df_complete = pd.read_csv(info_stage_complete_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
display_df(stage_df_complete)

,infoStage,notes_stage,stageId_,stageName_,stageAnnotationStatus_,infoStage_new,comment_stage
0,2-cell,Embryonic cleavage stage defined by a dividing 2-4 cells egg,MmusDv:0000005,Theiler stage 02,missing child term,2-cell (39-42h),
1,4-cell,Embryonic cleavage stage defined by a dividing 2-4 cells egg,MmusDv:0000005,Theiler stage 02,missing child term,4-cell (54-56h),
2,8-cell,"Embryonic cleavage stage defined by a dividing 4-16 cells egg, early to fully compacted morula.",MmusDv:0000006,Theiler stage 03,missing child term,8-cell (68-70h),
3,Blastocyst,Embryonic blastula stage defined by a zona free blastocyst (zona pellucida absent). Embryonic age 4 dpc (day post coitum).,MmusDv:0000009,Theiler stage 05,missing child term,Blastocyst (92-94h),
4,Inner cell mass,"Embryonic stage that is an early stage of embryonic development in animals and is produced by cleavage of a fertilized ovum, with formation of a central fluid-filled cavity called the blastocoel.",MmusDv:0000007,blastula stage,other,Inner cell mass,
5,MII oocyte,"mature stage definition: Mouse developmental stage that refers to a sexually mature adult mouse which is over 6 weeks old. Prime adult is a subset of mature stage, which is applicable here because we know the approximate age of the female",MmusDv:0000136,prime adult stage,other,MII oocyte (taken from 5- to 6-week-old female),
6,Zygote,Embryonic stage defined by a one-cell embryo (fertilised egg) with zona pellucida present,MmusDv:0000003,Theiler stage 01,perfect match,Zygote,
7,early 2-cell,Embryonic cleavage stage defined by a dividing 2-4 cells egg,MmusDv:0000005,Theiler stage 02,missing child term,early 2-cell (30_h),


In [14]:
library = library.merge(stage_df_complete, left_on='infoStage', right_on='infoStage')

In [15]:
library['stageId'] = library['stageId_'].values
library['stageName'] = library['stageName_'].values
library['stageAnnotationStatus'] = library['stageAnnotationStatus_'].values

In [16]:
## optional - update infoOrgan, add anat-specific comments 
library['infoStage'] = library['infoStage_new'].values
#library['comment'] = library[['comment', 'comment_stage']].agg(' '.join, axis=1)

In [17]:
# reformat, save progress
library = library[library_cols]
display_df(library)
library.to_csv(library_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,,,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,,17/10/2024
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,,,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,,17/10/2024
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,,,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,,17/10/2024
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,,,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,,17/10/2024
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,,,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,,17/10/2024
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,,,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,,17/10/2024
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,,,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,,other,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,,17/10/2024
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,,,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,,other,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,,17/10/2024
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBERON:0000358,blastocyst,,,MmusDv:0000009,Theiler stage 05,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1625877,Preimplantation embryo: Blastocyst,Blastocyst,,,,Blastocyst (92-94h),perfect match,,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq blastocyst KD control rep3,"SAMN03389622,GSM1625877",,,,17/10/2024
9,SRX902565,SRP055882,Illumina HiSeq 1500,SRS865219,UBE

#### cell type - smart-seq

In [18]:
cell_type = library['infoCellType_abInitio'].drop_duplicates().sort_values()
cell_df = pd.DataFrame(cell_type)
cell_df = cell_df.reindex(columns=[*cell_df.columns.tolist(), 'notes_cell', 'cellTypeId_', 'cellTypeName_', 'cellTypeAnnotationStatus_', 'cell_type_new','comment_cell'], fill_value="")
cell_df.to_csv(info_celltype_path, sep="\t", index=False)
if not os.path.exists(info_celltype_complete_path): 
    cell_df.to_csv(info_celltype_complete_path, sep="\t", index=False)

In [19]:
cell_df_complete = pd.read_csv(info_celltype_complete_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
display_df(cell_df_complete)

,infoCellType_abInitio,notes_cell,cellTypeId_,cellTypeName_,cellTypeAnnotationStatus_,cell_type_new,comment_cell
0,2-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,
1,4-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,
2,8-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,
3,Blastocyst,"using this general term for blastocyst cell, as we did in annotation of SRP020490. not using blastoderm cell because per the blastocyst definition: he mammalian blastocyst is a hollow ball of cells containing two cell types, the inner cell mass and the trophectoderm",CL:0002321,embryonic cell (metazoa),missing child term,,
4,Inner cell mass,Any native cell that is part of a inner cell mass.,CL:2000020,inner cell mass cell,perfect match,,
5,MII oocyte,"egg cell definition: A female gamete where meiosis has progressed to metaphase II and is able to participate in fertilization. MII oocyte is a metaphase II stage oocyte, therefore this annotation is appropriate.",CL:0000025,egg cell,perfect match,,
6,Zygote,,CL:0000365,animal zygote,perfect match,,
7,early 2-cell,An undifferentiated cell produced by early cleavages of the fertilized egg (zygote).,CL:0000353,blastoderm cell,perfect match,,


In [20]:
library = library.merge(cell_df_complete, left_on='infoCellType_abInitio', right_on='infoCellType_abInitio')

In [21]:
library['cellTypeId'] = library['cellTypeId_'].values
library['cellTypeName'] = library['cellTypeName_'].values
library['cellTypeAnnotationStatus'] = library['cellTypeAnnotationStatus_'].values

In [ ]:
## optional - update infoOrgan, add anat-specific comments 
#library['infoCellType_inferred'] = library['cell_type_new'].values
#library['comment'] = library[['comment', 'comment_cell']].agg(' '.join, axis=1)

In [22]:
# reformat, save progress
library = library[library_cols]
display_df(library)
library.to_csv(library_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,,17/10/2024
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,,17/10/2024
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,,17/10/2024
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,,17/10/2024
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,,17/10/2024
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,,C57BL/6N x DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,,17/10/2024
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,,17/10/2024
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,,17/10/2024
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBERON:0000358,blastocyst,CL:0002321,embryonic cell (metazoa),MmusDv:0000009,Theiler stage 05,http://www.ncbi.nlm.

#### sex, strain, genotype, speciesId
- uniprot [strain list](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/strains)
- uniprot [species list](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/speclist)
- bgee [strain mapping](https://gitlab.sib.swiss/Bgee/expression-annotations/-/tree/develop/Strains?ref_type=heads)
- sex options: M (male), F (female), NA (not available, unknown), mixed (both male and female)

In [25]:
library.loc[:,'sex'] = "NA"
library.loc[library["infoOrgan"] == "MII oocyte", "sex"] = "F"

#just updating the x to capital here
library.loc[library["strain"] == "C57BL/6N x DBA/2N", "strain"] = "C57BL/6N X DBA/2N"

#library.loc[:,'genotype'] = ''

#library.loc[:,'speciesId'] = ''

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,,17/10/2024
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,,17/10/2024
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,,17/10/2024
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,,17/10/2024
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,,17/10/2024
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,,17/10/2024
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,,17/10/2024
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,,17/10/2024
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBERON:0000358,blastocyst,CL:0002321,embryonic cell (metazoa),MmusDv:0000009,Theiler stage 05,http:/

#### protocol & RNAseqTags


In [26]:
# making these variables because we use them again in the experiment file
# usually 10X Genomics V1/V2/V3/V3.1, Smart-seq, Smart-seq2
my_protocol = 'Smart-Seq2'
# Full-length or 3'end or 5'end
my_protocolType = "Full-length"

library.loc[:,'protocol'] = my_protocol
library.loc[:,'protocolType'] = my_protocolType
# scRNA-seq, Sn-scRNA-seq
library.loc[:,'RNAseqTags'] = 'scRNA-seq'

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,,17/10/2024
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,,17/10/2024
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,,17/10/2024
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,,17/10/2024
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,,17/10/2024
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,,17/10/2024
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,,17/10/2024
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,,17/10/2024
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBERON:0000358,blastocyst,CL

#### condition

In [ ]:
# ex. control, diet, light, reproductive capacity, time post mortem, time post feeding, 
# exercise details, menstruation, personality, litter size 
#library.loc[library["condition"] == "old", "condition"] = "new"

# view
display_df(library)

#### annotator id, last modification date

In [27]:
library.loc[:,'annotatorId'] = 'SAC'
library.loc[:,'lastModificationDate'] = '2024-10-22'

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,SAC,2024-10-22
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,SAC,2024-10-22
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,SAC,2024-10-22
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,SAC,2024-10-22
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,SAC,2024-10-22
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,SAC,2024-10-22
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,SAC,2024-10-22
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,SAC,2024-10-22
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBER

#### comments

In [ ]:
#library.loc[:,'comment'] = ''

#### save complete file with correct columns

In [28]:
library_file_complete = library[library_cols]

library_file_complete.to_csv(library_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

# view
display_df(library_file_complete)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
0,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933942,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM1933942",,,SAC,2024-10-22
1,SRX1424869,SRP055882,Illumina HiSeq 1500,SRS1157158,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933941,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep3,"SAMN04252375,GSM1933941",,,SAC,2024-10-22
2,SRX1424868,SRP055882,Illumina HiSeq 1500,SRS1157157,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933940,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep2,"SAMN04252374,GSM1933940",,,SAC,2024-10-22
3,SRX1424867,SRP055882,Illumina HiSeq 1500,SRS1157161,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933939,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep1,"SAMN04252373,GSM1933939",,,SAC,2024-10-22
4,SRX1424866,SRP055882,Illumina HiSeq 1500,SRS1157160,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933938,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep2,"SAMN04252372,GSM1933938",,,SAC,2024-10-22
5,SRX1424865,SRP055882,Illumina HiSeq 1500,SRS1157159,UBERON:0019249,2-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000005,Theiler stage 02,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933937,Preimplantation embryo: early 2-cell,early 2-cell,,,,early 2-cell (30_h),missing child term,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq early 2-cell rep1,"SAMN04252371,GSM1933937",,,SAC,2024-10-22
6,SRX1424864,SRP055882,Illumina HiSeq 1500,SRS1157162,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933936,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep2,"SAMN04252370,GSM1933936",,,SAC,2024-10-22
7,SRX1424863,SRP055882,Illumina HiSeq 1500,SRS1157163,CL:0000025,egg cell,CL:0000025,egg cell,MmusDv:0000136,prime adult stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1933935,MII oocyte,MII oocyte,,,,MII oocyte (taken from 5- to 6-week-old female),perfect match,perfect match,other,F,C57BL/6N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq MII oocyte rep1,"SAMN04252369,GSM1933935",,,SAC,2024-10-22
8,SRX902566,SRP055882,Illumina HiSeq 1500,SRS865220,UBER

### experiment annotations

In [29]:
experiment = pd.read_csv(experiment_path_from_script, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
experiment.head()

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,RNAseqTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP055882,The landscape of accessible chromatin in mamma...,The landscape of accessible chromatin in mamma...,SRA,,,,"Smart-Seq2, ATAC-Seq","Full-length, 3'end",GSE66582,PRJNA277361,27309802,,"E,r,r,o,r,:, ,U,n,a,b,l,e, ,t,o, ,r,e,t,r,i,e,...",,


#### experiment and protocol details

In [30]:
library_to_add = pd.read_csv(library_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)

# this will give you the number of rows in the complete library file 
# this should be the number of annotated libraries
ann_lib = len(library_to_add.index)
len(library_to_add.index)

22

In [31]:
# partial or total
experiment.loc[:,'experimentStatus'] = 'partial'
# scRNA-seq, Sn-scRNA-seq for both we put: 'Sn-scRNA-seq, scRNA-seq'
experiment.loc[:,'RNAseqTags'] = 'scRNA-seq'


# see above cell, also can add as free text
experiment.loc[:,'numberOfAnnotatedLibraries'] = ann_lib

# these variables should already exist from above but if not can just add as free text
experiment.loc[:,'protocol'] = my_protocol
experiment.loc[:,'protocolType'] = my_protocolType

experiment.head()

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,RNAseqTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP055882,The landscape of accessible chromatin in mamma...,The landscape of accessible chromatin in mamma...,SRA,partial,scRNA-seq,22,Smart-Seq2,Full-length,GSE66582,PRJNA277361,27309802,,"E,r,r,o,r,:, ,U,n,a,b,l,e, ,t,o, ,r,e,t,r,i,e,...",,


#### paper and xrefs

In [32]:
#experiment.loc[:,'GSE'] = ''
#experiment.loc[:,'Bioproject'] = '' 
#experiment.loc[:,'PMID'] = ''
experiment.loc[:,'reference_url'] = 'https://www.nature.com/articles/nature18606'
experiment.loc[:,'DOI'] = '10.1038/nature18606'
#experiment.loc[:,'xrefs'] = ''

experiment.head()

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,RNAseqTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP055882,The landscape of accessible chromatin in mamma...,The landscape of accessible chromatin in mamma...,SRA,partial,scRNA-seq,22,Smart-Seq2,Full-length,GSE66582,PRJNA277361,27309802,https://www.nature.com/articles/nature18606,10.1038/nature18606,,


#### comments

In [33]:
experiment.loc[:,'comment'] = 'partial - removed cell culture and siRNA knock-down libraries'

experiment.head()

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,RNAseqTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP055882,The landscape of accessible chromatin in mamma...,The landscape of accessible chromatin in mamma...,SRA,partial,scRNA-seq,22,Smart-Seq2,Full-length,GSE66582,PRJNA277361,27309802,https://www.nature.com/articles/nature18606,10.1038/nature18606,,partial - removed cell culture and siRNA knock...


#### save complete file

In [34]:
experiment.to_csv(experiment_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

### QA time

In [35]:
library_to_add = pd.read_csv(library_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
experiment_to_add = pd.read_csv(experiment_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)

#### to add things here

#### check columns match

In [36]:
# pull from git and pull in library/experiment file
! git pull
git_library = pd.read_csv(git_library_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)
git_experiment = pd.read_csv(git_experiment_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN', " "], dtype=object)

# library file
if set(library_to_add.columns) == set(git_library.columns):
    print('The columns in the library file match')
else:
    print('The columns in the library file DO NOT MATCH')

# experiment file
if set(experiment_to_add.columns) == set(git_experiment.columns):
    print('The columns in the experiment file match')
else:
    print('The columns in the experiment file DO NOT MATCH')


# maybe to make this something more like "COLUMNS GOOD - LIBRARY" and "COLUMNS BAD - EXPERIMENT"

Already up to date.
The columns in the library file match
The columns in the experiment file match


#### view files

In [37]:
library_git_plus_new = pd.concat([git_library, library_to_add], ignore_index = True, sort = False)
library_git_plus_new.tail(n=30)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,cellTypeId,cellTypeName,stageId,stageName,url_GSM,infoOrgan,infoCellType_abInitio,infoCellType_inferred,clusterId,clusterName,infoStage,anatAnnotationStatus,cellTypeAnnotationStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,RNAseqTags,protocol,protocolType,lib_name,sampleName,comment,condition,annotatorId,lastModificationDate
58223,SRX23721833,SRP491411,Illumina NovaSeq 6000,SRS20552356,UBERON:0016534,white matter of temporal lobe,,,HsapDv:0000118,24-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe - white matter,Endothelial cells,,,,24 years,missing child term,,perfect match,M,,,9606,scRNA-seq,10X Genomics V2,3'end,TL_24-02-2020_whitematter_sorted_3pr_v3,SAMN40082474,,,SAC,2024-09-30
58224,SRX23721832,SRP491411,Illumina NovaSeq 6000,SRS20549927,UBERON:0016538,temporal cortex,,,HsapDv:0000118,24-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe - grey matter,Endothelial cells,,,,24 years,missing child term,,perfect match,M,,,9606,scRNA-seq,10X Genomics V2,3'end,TL_24-02-2020_greymatter_sorted_2ndbatch_3pr_v3,SAMN40082475,,,SAC,2024-09-30
58225,SRX23721831,SRP491411,Illumina NovaSeq 6000,SRS20551510,UBERON:0000955,brain,,,HsapDv:0000026,Carnegie stage 19,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,brain - CNS,"Endothelial, perivascular and other tissue der...",,,,9.1 gestational weeks,perfect match,,missing child term,NA,,,9606,scRNA-seq,10X Genomics V2,3'end,9point1_Weeks_Fetal_CNS_UNSORTED_3pr_v3_1,SAMN40082467,,,SAC,2024-09-30
58226,SRX23721826,SRP491411,Illumina NovaSeq 6000,SRS20552352,UBERON:0002809,right temporal lobe,,,HsapDv:0000133,39-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe,Endothelial cells,,,,39 years,perfect match,,perfect match,F,,,9606,scRNA-seq,10X Genomics V2,3'end,TL7,SAMN40082476,,,SAC,2024-09-30
58227,SRX23721825,SRP491411,Illumina NovaSeq 6000,SRS20551508,UBERON:0016534,white matter of temporal lobe,,,HsapDv:0000119,25-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe - white matter,"Endothelial, perivascular and other tissue der...",,,,25 years,missing child term,,perfect match,M,,,9606,scRNA-seq,10X Genomics V2,3'end,TL_20-02-2020_whitematter_unsorted_3pr_v3,SAMN40082477,,,SAC,2024-09-30
58228,SRX23721824,SRP491411,Illumina NovaSeq 6000,SRS20552354,UBERON:0016538,temporal cortex,,,HsapDv:0000119,25-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe - grey matter,"Endothelial, perivascular and other tissue der...",,,,25 years,missing child term,,perfect match,M,,,9606,scRNA-seq,10X Genomics V2,3'end,TL_20-02-2020_greymatter_unsorted_3pr_v3,SAMN40082478,,,SAC,2024-09-30
58229,SRX23721823,SRP491411,Illumina NovaSeq 6000,SRS20552353,UBERON:0016534,white matter of temporal lobe,,,HsapDv:0000119,25-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe - white matter,Endothelial cells,,,,25 years,missing child term,,perfect match,M,,,9606,scRNA-seq,10X Genomics V2,3'end,TL_20-02-2020_whitematter_sorted_3pr_v3,SAMN40082479,,,SAC,2024-09-30
58230,SRX23721822,SRP491411,Illumina NovaSeq 6000,SRS20552141,UBERON:0016538,temporal cortex,,,HsapDv:0000119,25-year-old stage,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,right temporal lobe - grey matter,Endothelial cells,,,,25 years,missing child term,,perfect match,M,,,9606,scRNA-seq,10X Genomics V2,3'end,TL_20-02-2020_greymatter_sorted_3pr_v3,SAMN40082480,,,SAC,2024-09-30
58231,SRX1424870,SRP055882,Illumina HiSeq 1500,SRS1157155,UBERON:0019252,8-cell stage embryo,CL:0000353,blastoderm cell,MmusDv:0000006,Theiler stage 03,http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?...,Preimplantation embryo: 8-cell,8-cell,,,,8-cell (68-70h),perfect match,perfect match,missing child term,NA,C57BL/6N X DBA/2N,,10090,scRNA-seq,Smart-Seq2,Full-length,RNA-Seq 8-cell KD control rep4,"SAMN04252376,GSM19339

In [38]:
experiment_git_plus_new = pd.concat([git_experiment, experiment_to_add], ignore_index = True, sort = False)
experiment_git_plus_new.tail(n=5)

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,RNAseqTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
99,SRP192549,Single-cell transcriptomic profiling of the ag...,"The mammalian brain is complex, with multiple ...",SRA,total,scRNA-seq,16 libraries,10X Genomics V2,3'end,GSE129788,PRJNA532831,31551601,https://www.nature.com/articles/s41593-019-0491-3,10.1038/s41593-019-0491-3,https://singlecell.broadinstitute.org/single_c...,annotation of cell types may be improved in th...
100,SRP125125,RNA-Seq profiling of 29 immune cell types and ...,We performed RNA-Seq transcriptome profiling o...,SRA,total,scRNA-seq,127 libraries,Adapted Smart-Seq2,Full-length,GSE107011,PRJNA418779,30726743,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,10.1016/j.celrep.2019.01.041,30528453[PMID],"NTR to check for annotation improvement, see h..."
101,SRP356459,Molecular atlas of the human brain vasculature...,Molecular atlas of the human brain vasculature...,SRA,partial,scRNA-seq,27,10X Genomics V2,3'end,GSE256490,PRJNA800024,38987604,https://www.nature.com/articles/s41586-024-074...,10.1038/s41586-024-07493-y,,"GEO says 10X Genomics V2, supplemental methods..."
102,SRP491411,Single-cell atlas of the human brain vasculatu...,A broad range of brain pathologies critically ...,SRA,partial,scRNA-seq,11,10X Genomics V2,3'end,GSE256491,PRJNA1079775,38987604,https://www.nature.com/articles/s41586-024-074...,10.1038/s41586-024-07493-y,,"GEO says 10X Genomics V2, supplemental methods..."
103,SRP055882,The landscape of accessible chromatin in mamma...,The landscape of accessible chromatin in mamma...,SRA,partial,scRNA-seq,22,Smart-Seq2,Full-length,GSE66582,PRJNA277361,27309802,https://www.nature.com/articles/nature18606,10.1038/nature18606,,partial - removed cell culture and siRNA knock...


### add annotations to git

In [39]:
! git pull

Already up to date.


In [40]:
library_git_plus_new.to_csv(git_library_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
update_format(git_library_path)
experiment_git_plus_new.to_csv(git_experiment_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
update_format(git_experiment_path)

In [41]:
! git status

On branch develop
Your branch is up to date with 'origin/develop'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../1_sara_scRNA_template.ipynb
	modified:   ../../../scRNA_Seq/scRNASeqExperiment.tsv
	modified:   ../../../scRNA_Seq/scRNASeqLibrary_merged.tsv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	./

no changes added to commit (use "git add" and/or "git commit -a")


In [42]:
# smart-seq (no barcode file)
! git add $git_experiment_path $git_library_path

In [43]:
! git status

On branch develop
Your branch is up to date with 'origin/develop'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   ../../../scRNA_Seq/scRNASeqExperiment.tsv
	modified:   ../../../scRNA_Seq/scRNASeqLibrary_merged.tsv

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../1_sara_scRNA_template.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	./



In [44]:
! git commit -m $commit_message_exp

[develop 100dd90] adding annotated scRNA experiment SRP055882
 2 files changed, 23 insertions(+)


In [45]:
! git push

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 12 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 2.58 KiB | 2.58 MiB/s, done.
Total 5 (delta 4), reused 0 (delta 0), pack-reused 0
remote: 
remote: To create a merge request for develop, visit:
remote:   https://gitlab.sib.swiss/Bgee/expression-annotations/-/merge_requests/new?merge_request%5Bsource_branch%5D=develop
remote: 
To https://gitlab.sib.swiss/Bgee/expression-annotations.git
   2ca23c3..100dd90  develop -> develop


### add annotation folder and script to git

first - run annotation summary cells, save ipynb, export to html 

In [ ]:
! git status

In [ ]:
! git add $path_to_output

In [ ]:
! git commit -m $commit_message_py

In [ ]:
! git push